In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.stats import zscore

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv
/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv


In [16]:
train_data = pd.read_csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv')
test_data = pd.read_csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv')

In [17]:
train_data['date'] = pd.to_datetime(train_data['date']).dt.strftime('%Y%m%d')
test_data['date'] = pd.to_datetime(test_data['date']).dt.strftime('%Y%m%d')

In [18]:
total_size = len(train_data)
split_ratio = .8
split_size = int(total_size * split_ratio)
train_set = train_data[0:split_size]
val_set = train_data[split_size:]


In [19]:
# # Plot the data
# plt.plot(train_set['date'], train_set['meantemp'])
# plt.plot(val_set['date'], val_set['meantemp'])

# # Set the plot title and axis labels
# plt.title('train and val split ')
# plt.xlabel('date')
# plt.ylabel('meantemp')

# # Show the plot
# plt.show()

In [20]:
# columns_to_convert = ['humidity', 'wind_speed', 'meanpressure', 'meantemp']
columns_to_convert = ['meantemp']

train_set_cols = train_set[columns_to_convert]
val_set_cols = val_set[columns_to_convert]

train_data_normalized = train_set_cols.apply(zscore)
val_data_normalized = val_set_cols.apply(zscore)

In [21]:
train_set_array = train_data_normalized.to_numpy()
val_set_array = val_data_normalized.to_numpy()


In [23]:
# constants
window_size = 64
batch_size = 256
shuffle_buffer_size = 1000

In [24]:
# def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
#     # expand dims to include all columns
#     series = tf.expand_dims(series, axis=-1)
#     ds = tf.data.Dataset.from_tensor_slices(series)
#     ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
#     ds = ds.flat_map(lambda w: w.batch(window_size + 1))
#     ds = ds.shuffle(shuffle_buffer)
    
#     # modify map function to include all columns
#     ds = ds.map(lambda window: (window[:-1], window[-1]))
    
#     return ds.batch(batch_size).prefetch(1)
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # YOUR CODE HERE
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [25]:
train_window_set = windowed_dataset(train_set_array, window_size, batch_size, shuffle_buffer_size)
val_window_set = windowed_dataset(val_set_array, window_size, batch_size, shuffle_buffer_size)

In [26]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
model = tf.keras.models.Sequential([
    # YOUR CODE HERE
  tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                           strides=1, padding="causal",
                           activation="relu",
                           input_shape=[None, 1]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),    
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])


optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)# YOUR CODE HERE, momentum=0.9)
model.compile(loss='mae',
              optimizer='adam',
              metrics=["mae", "mse", "mape"])
history = model.fit(train_window_set,epochs=20)# YOUR CODE HERE)

Epoch 1/20
5/5 [==============================] - 15s 506ms/step - loss: 0.8547 - mae: 0.8560 - mse: 1.0365 - mape: 143.3371
Epoch 2/20
5/5 [==============================] - 3s 509ms/step - loss: 0.8460 - mae: 0.8438 - mse: 1.0162 - mape: 151.4795
Epoch 3/20
5/5 [==============================] - 3s 616ms/step - loss: 0.8434 - mae: 0.8398 - mse: 1.0254 - mape: 159.9624
Epoch 4/20
5/5 [==============================] - 3s 520ms/step - loss: 0.8436 - mae: 0.8456 - mse: 1.0312 - mape: 156.1198
Epoch 5/20
5/5 [==============================] - 3s 529ms/step - loss: 0.8443 - mae: 0.8430 - mse: 1.0385 - mape: 163.8758
Epoch 6/20
5/5 [==============================] - 3s 520ms/step - loss: 0.8465 - mae: 0.8468 - mse: 1.0017 - mape: 142.3689
Epoch 7/20
5/5 [==============================] - 3s 535ms/step - loss: 0.8447 - mae: 0.8447 - mse: 1.0085 - mape: 147.5095
Epoch 8/20
5/5 [==============================] - 3s 525ms/step - loss: 0.8438 - mae: 0.8459 - mse: 1.0679 - mape: 171.3163
Epoch 9